In [ ]:
pip install IMDbPY
pip install emoji
pip install fpdf

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from fpdf import FPDF
import re
from datetime import datetime
from functools import reduce
API_KEY = "89b6d074"

In [18]:
import requests
from bs4 import BeautifulSoup
import csv
from fpdf import FPDF
import re
from datetime import datetime  # ✅ Added import for timestamp
from functools import reduce
API_KEY = "89b6d074"

#---------------------- #
#1️⃣ Web Scraping Phase  #
#---------------------- #
def fetch_movies_from_rotten_tomatoes():
    print("🍅 Fetching Latest Movies from Rotten Tomatoes...")
    url = "https://www.rottentomatoes.com/browse/movies_in_theaters"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers,verify=False)
        soup = BeautifulSoup(response.content, "html.parser")

        titles = [tag.get("href").split("/")[-1].replace("-", " ").title() 
                  for tag in soup.select('a.js-tile-link')]

        print(f"✅ Fetched {len(titles)} movies from Rotten Tomatoes!\n")
        return titles[:30]

    except requests.exceptions.RequestException as e:
        print(f"❌ Error while scraping Rotten Tomatoes: {e}")
        return []

titles = ['Detective_Chinatown_1900', 'Hit_N_Fun', 'Gi_Dle_World_Tour_Idol_In_Cinemas', 'Round_The_Decay', 'Ponman', 'Thudarum', 'No_Other_Land', 'Screamboat', 'Picnic_At_Hanging_Rock', 'Marcello_Mio', 'Time_Passages', 'Majhail', 'When_We_Free_The_World']
# ---------------------- #
# 2️⃣ Extract Phase       #
# ---------------------- #
def extract_movies(titles):
    print("🚀 Fetching Data from OMDb API... 🎬")
    movies = []

    for title in titles:
        url = f"http://www.omdbapi.com/?t={title}&apikey={API_KEY}"
        try:
            response = requests.get(url, timeout=10)
            data = response.json()

            if data.get("Response") == "True":
                movies.append(data)
            else:
                print(f"❌ Movie not found: {title}")

        except requests.exceptions.RequestException as e:
            print(f"❌ Request Error for {title}: {e}")

    print(f"✅ Fetched {len(movies)} movies successfully!\n")
    return movies

# ---------------------- #
# 3️⃣ Transform Phase      #
# ---------------------- #
def transform_data(movies):
    print("🔄 Transforming Below Data... for clean up🧹✨")
    transformed_movies = []  # ✅ Initialize the list

    for movie in movies:
         #TASK1 - Title Transformation ( •	Remove Special Characters: Use regular expressions to eliminate non-alphanumeric characters.)
           #( •	Standardized Case: Convert all titles to Title Case for consistency.)
        #(•	Trim Whitespaces: Remove leading/trailing spaces to maintain uniformity.)

        title_clean = re.sub(r'[^\w\s]', '', movie.get("Title", "").strip()).title() 
        print(title_clean)
        
        #TASK2 - Release Date Transformation (•	Date Formatting: Replace spaces with hyphens to standardize the date format (e.g., DD MMM YYYY to DD-MMM-YYYY).
            #(•	Convert to Date Object: Use date parsing to convert text dates into proper date formats.)
            #(•	Handle Missing Dates: Replace missing dates with "Unknown.")

        release_date = movie.get("Released", "N/A")   
        formatted_date = (
            datetime.strptime(release_date, '%d %b %Y').strftime('%Y-%m-%d') 
            if release_date != "N/A" else "Unknown"
        )
        print(release_date)
         #TASK3 - Genre Transformation ( •Convert to Lowercase: Ensure all genres are in lowercase for consistency.)
                     #(•Split Genres: If multiple genres are present, separate them into a list.)
                     #(•Remove Duplicates: Ensure unique genre entries.)
        genres = list(set([genre.strip().lower() for genre in movie.get("Genre", "").split(",")])) 
        print(genres)

        #TASK4- IMBD Rating Transformation (•Convert to Numeric: Change rating from text to a floating-point number.)
                    #(•	Round Off: Round ratings to one decimal place.)
                    #(•	Normalize: Optionally, normalize ratings on a scale of 0 to 1.)

        imdb_rating = round(float(movie.get("imdbRating", 0)), 2) if movie.get("imdbRating") != "N/A" else None  
        imdb_rating_normalized = round(imdb_rating / 10,2) if imdb_rating!=None else "No Rating"
        print(imdb_rating_normalized)

         #TASK5- Actors Transformation   (•	Limit to Top 3: Display only the top three actors.)
                    #(•	Trim Spaces: Remove extra spaces around names.)
                    #(•	Sort Alphabetically (Optional): For consistency in display.)

        actors = ", ".join(sorted([actor.strip() for actor in movie.get("Actors", "").split(",")[:3]]))
        print(actors)

         #TASK6 - BoxOffice Transofrmation   (•	Remove Currency Symbols: Eliminate $, ,, and other non-numeric characters.)
                    #(•	Convert to Numeric: Store as an integer for analysis.)
                    #(•	Handle Missing Data: Replace missing values with 0 or N/A)

        box_office_clean = int(re.sub(r'[^\d]', '', movie.get("BoxOffice", ""))) if movie.get("BoxOffice","")!="N/A" else "No Collection Reported"
        print(box_office_clean)
        box_office_value = box_office_clean
        # print(box_office_clean)

        # TASK7- Awards Transformation    (•	Extract Numbers: Identify and sum all numeric values related to awards won.)
        #             (•	Standardize Format: Display total awards won.)
        #             (•	Handle Missing Awards: Set to 0 if no data is available.)

        awards_split = movie.get("Awards", "").split('&') if movie.get("Awards","")!="N/A" else None
        if awards_split != None:
            print(awards_split[0])
            awards = re.findall(r'\d+', awards_split[0])
            # nominations = re.findall(r'\d+', awards_split[1])
            print(awards)
            # print(nominations)
            total_awards = reduce(lambda x,y: x+y, awards)
            # nomi_total = reduce(lambda x,y: x+y, nominations)
            print(f"Total awards are {total_awards}") #and Total Nominations are {nomi_total}" )
            
        #TASK8- MetasScore Transformation  (•Convert to Integer: Change metascore to an integer for calculations.)
                    #(•	Normalize: Convert to a 0-1 scale by dividing by 100.)
                    #(•	Handle Missing Values: Replace "N/A" with None.)

        metascore = int(movie.get("Metascore",""),0) if movie.get("Metascore")!="N/A" else None
        metascore_normalized = metascore / 100 if metascore!= None else "No Metascore"
        print(metascore_normalized)

        #TASK9 - Language Transformation  (•Convert to Lowercase: Ensure all language names are in lowercase.)
                    #(•	Standardize Codes: Optionally convert to ISO language codes.)
                   #(•	Handle Missing Data: Replace missing languages with "Unknown.")

        language = movie.get("Language","") if movie.get("Language")!="N/A" else "Unkown"
        print(language.lower())

        #TASK10 - Production Transformation  (•	Remove Special Characters: Clean the production company names.)
                #(•	Standardize Names: Correct common misspellings and standardize abbreviations.)
                #(•	Handle Missing Data: Replace missing production companies with "Independent.")

        production = re.sub(r'[^\w\s]', '', movie.get("Production", "").strip()).title() if movie.get("Production","")!="N/A" else "Independent"
        print(movie.get("Production",""))

        transformed_movies.append({
            "Title": title_clean,
            "Release Date": formatted_date,
            "Genres": genres,
            "IMDb Rating": imdb_rating,
            "Normalized IMDb Rating": imdb_rating_normalized,
            "Top 3 Actors": actors,
            "Box Office": box_office_value,
            "Total Awards": total_awards,
            "Metascore": metascore_normalized,
            "Language": language,
            "Production": production
        })

    print(f"✨ Transformation Complete for {len(transformed_movies)} movies!\n")
    return transformed_movies

# ---------------------- #
# 4️⃣ Load Phase (CSV)     #
# ---------------------- #
def load_to_csv(data):
    print("💾 Saving Data to CSV... 📊")
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"movies_{timestamp}.csv"  # ✅ Add timestamp to avoid overwrite

    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)

    print(f"✅ Data successfully saved to {filename}!\n")


# ---------------------- #
# 🚀 Execute ETL Pipeline #
# ---------------------- #
if __name__ == "__main__":
    titles = fetch_movies_from_rotten_tomatoes()
    if titles:
        movies = extract_movies(titles)
        if movies:
            transformed_data = transform_data(movies)
            load_to_csv(transformed_data)
            
        else:
            print("⚠️ No movie data fetched from the API.")
    else:
        print("⚠️ No movie titles found during scraping.")


🍅 Fetching Latest Movies from Rotten Tomatoes...


C:\Users\SMangal\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.rottentomatoes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ Fetched 10 movies from Rotten Tomatoes!

🚀 Fetching Data from OMDb API... 🎬
❌ Movie not found: The_Visitor_2024
❌ Movie not found: Music_Night_Live_From_Nashville
❌ Movie not found: Ufc_313_Pereira_Vs_Ankalaev
✅ Fetched 7 movies successfully!

🔄 Transforming Below Data... for clean up🧹✨
Purple Rain
27 Jul 1984
['drama', 'romance', 'music']
0.65
Apollonia Kotero, Morris Day, Prince
70261052
Won 1 Oscar. 9 wins 
['1', '9']
Total awards are 19
0.55
english
N/A
Guns Of Redemption
07 Mar 2025
['western']
No Rating
Casper Van Dien, Kaitlyn Kemp, Siena Bjornerud
No Collection Reported
No Metascore
english
N/A
The Island Between Tides
07 Mar 2024
['drama', 'mystery']
No Rating
David Mazouz, Donal Logue, Paloma Kwiatkowski
No Collection Reported
No Metascore
english
N/A
Play It As It Lays
07 Jun 1973
['drama']
0.64
Anthony Perkins, Tammy Grimes, Tuesday Weld
No Collection Reported
1 win 
['1']
Total awards are 1
No Metascore
english
N/A
The Way My Way
16 May 2024
['drama']
0.67
Chris Haywood,

#### The following code is optional - just for learning purposed. The following code is to embeds the generated csv file into a pdf

In [19]:
from fpdf import FPDF
from datetime import datetime

def load_to_pdf(data):
    print("📝 Generating PDF Report... 📄")
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"movies_{timestamp}.pdf"

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # ✅ Title
    pdf.cell(200, 10, "Movie Insights Report", ln=True, align="C")
    pdf.ln(5)

    # 🔹 Add Data with Bifurcation Lines
    for movie in data:
        pdf.set_font("Arial", style='B', size=12)  # Bold for Title
        pdf.cell(0, 8, f" {movie.get('Title', 'N/A')}", ln=True)
        pdf.set_font("Arial", size=11)

        for key, value in movie.items():
            if key != "Title":  # Skip repeating the title
                pdf.multi_cell(0, 8, f"{key}: {value}")

        # 🚀 Add a Bifurcation Line for Separation
        pdf.set_draw_color(0, 0, 0)  # Black color
        pdf.set_line_width(0.5)
        pdf.line(10, pdf.get_y(), 200, pdf.get_y())  # Draw horizontal line
        pdf.ln(5)  # Add space after the line

    pdf.output(filename)
    print(f"✅ PDF report saved as {filename}!\n")
load_to_pdf(transformed_data)

📝 Generating PDF Report... 📄
✅ PDF report saved as movies_20250305_125746.pdf!

